In [1]:
#Import Dependencies
import datetime as dt
import numpy as np
import pandas as pd

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify, render_template, abort

In [2]:
# Create engine using the `hawaii.sqlite` database file
engine = create_engine("sqlite:///../Resources/hawaii.sqlite", echo=False)

In [3]:
#using automap to load orm automatically

Base = automap_base()


In [4]:
# reflecting the tables from orm classes

Base.prepare(engine, reflect = True)

Measurement = Base.classes.measurements
Station = Base.classes.stations
print(Base.classes.keys())

['measurements', 'stations']


In [5]:
#initiating session
session = Session(engine)


In [6]:
#Flask
from flask import Flask, jsonify

app = Flask(__name__)

In [7]:
@app.route("/")
def welcome(): 
    return (
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/stations</br>"
        f"/api/v1.0/<start></br>"
        f"/api/v1.0/<start>/<end>"
    )

In [8]:
@app.route("/api/v1.0/precipitation")
def precipitation():
    dict = {}
    data = df[(df.date >= str(prev_year)) & (df.date <= str(today))].groupby(['date']).sum()
    for index, row in df.iterrows():
        dict[row.date] = row.prcp
    return jsonify(dict)

In [9]:
# /api/v1.0/stations
# Return a json list of stations from the dataset.
@app.route("/api/v1.0/stations")
def stations():
    stations = list(df.measurement.unique())
    return jsonify(stations)

In [10]:
# /api/v1.0/tobs
# Return a json list of Temperature Observations (tobs) for the previous year
@app.route("/api/v1.0/tobs")
def tobs():
    results = session.query(measurement.date, measurement.tobs).filter(measurement.date >= prev_year).all()
    temp_results = list(np.ravel(results))

    return jsonify(temp_results)


In [11]:
@app.route("/api/v1.0/<start>")
def temp_start(start):
    temps = df[(df.date >= start)]
    min_temp = temps.tobs.min()
    max_temp = temps.tobs.max()
    mean_temp = temps.tobs.mean()
    return jsonify({"min_temp": int(min_temp), "max_temp": int(max_temp), "mean_temp": int(mean_temp)})

In [12]:
@app.route("/api/v1.0/<start>/<end>")
def temp_range(start, end):
    temps = df[(df.date >= start) & (df.date <= end)]
    min_temp = temps.tobs.min()
    max_temp = temps.tobs.max()
    mean_temp = temps.tobs.mean()
    return jsonify({"min_temp": int(min_temp), "max_temp": int(max_temp), "mean_temp": int(mean_temp)})

In [14]:
# With debug=True, Flask server will auto-reload 
# when there are code changes
if __name__ == '__main__':
	app.run(debug=False)

UnsupportedOperation: not writable